## Collecting song information

In [1]:
!pip install spotipy

In [2]:
import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [3]:
cid = "3519692942004325a2c7160c90717ca5"
secret = "9243be1df96e48bb829c4b07254bd82c"
redirect_uri = 'https://github.com/manav-s/audioalchemy'
scope = 'playlist-modify-public playlist-modify-private'

In [4]:
# client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
playlist_link = "https://open.spotify.com/playlist/1hH8ZCf9orRRymgRP95jXb?si=e45b5924a9c0435b&pt=e744ed4e7acc7c495604af2b8214e392"
uri = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(uri)["items"]]

In [6]:
print(sp.audio_features(track_uris)[0])

{'danceability': 0.559, 'energy': 0.556, 'key': 6, 'loudness': -7.844, 'mode': 1, 'speechiness': 0.531, 'acousticness': 0.553, 'instrumentalness': 0, 'liveness': 0.251, 'valence': 0.362, 'tempo': 189.861, 'type': 'audio_features', 'id': '7lL3MvFWFFSD25pBz72Agj', 'uri': 'spotify:track:7lL3MvFWFFSD25pBz72Agj', 'track_href': 'https://api.spotify.com/v1/tracks/7lL3MvFWFFSD25pBz72Agj', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7lL3MvFWFFSD25pBz72Agj', 'duration_ms': 269787, 'time_signature': 4}


In [7]:
import pandas as pd

# Initialize empty lists for columns
song_names = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

# Loop through the tracks and append the data to the respective lists
for track in sp.playlist_tracks(playlist_link)["items"]:
    track_name = track["track"]["name"]
    song_names.append(track_name)
    
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    danceability.append(audio_features["danceability"])
    energy.append(audio_features["energy"])
    key.append(audio_features["key"])
    loudness.append(audio_features["loudness"])
    mode.append(audio_features["mode"])
    speechiness.append(audio_features["speechiness"])
    acousticness.append(audio_features["acousticness"])
    instrumentalness.append(audio_features["instrumentalness"])
    liveness.append(audio_features["liveness"])
    valence.append(audio_features["valence"])
    tempo.append(audio_features["tempo"])

# Create a DataFrame
data = {
    "Song Name": song_names,
    "Danceability": danceability,
    "Energy": energy,
    "Key": key,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo
}

df = pd.DataFrame(data)
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080


# Score

We need a weighting system that identifies the importance of each feature. It should add up to 1.

In [8]:
def get_relative_key(key, mode):
    if mode == 1:  # Major key
        return (key + 9) % 12  # Relative minor key
    else:  # Minor key
        return (key + 3) % 12  # Relative major key

def key_compatibility(key1, mode1, key2, mode2):
    if key1 == key2:
        return True
    if mode1 != mode2 and (key1 == get_relative_key(key2, mode2) or key2 == get_relative_key(key1, mode1)):
        return True
    return False

def evaluate_transition(song1, song2):
    # Weights for different attributes
    weights = {
        'danceability': 1,
        'energy': 1,
        'loudness': 0.5,
        'tempo': 10,  # Increase the weight of tempo
        'valence': 1
    }

    # Check key compatibility
    if not key_compatibility(song1['key'], song1['mode'], song2['key'], song2['mode']):
        return float('inf')  # Return a large score if keys are not compatible

    # Calculate the differences in attributes
    diff = {}
    for attribute in weights:
        diff[attribute] = abs(song1[attribute] - song2[attribute])

    # Normalize loudness differences
    diff['loudness'] /= 60  # Assume max difference is 60 dB

    # Calculate tempo difference considering double/half time mixing
    tempo_diff = min(
        abs(song1['tempo'] - song2['tempo']),
        abs(song1['tempo'] * 2 - song2['tempo']),
        abs(song1['tempo'] / 2 - song2['tempo'])
    )
    diff['tempo'] = tempo_diff / 200  # Normalize tempo difference

    # Calculate the transition score
    score = 0
    for attribute in weights:
        score += weights[attribute] * diff[attribute]

    return score

## Implementing Beam Search

## Reordering the Spotify playlist

We will add the new index of each song to the dataframe

In [9]:
df['new_index'] = pd.Series(None, index=df.index)
df

<ipython-input-9-18d42472c2f2>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['new_index'] = pd.Series(None, index=df.index)


,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,new_index
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861,NaN
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103,NaN
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793,NaN
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983,NaN
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013,NaN
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827,NaN
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239,NaN
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080,NaN


In [10]:
# testing by reversing the index
df['new_index'] = df.index[::-1]
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,new_index
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861,99
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103,98
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793,97
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983,96
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013,4
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827,3
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239,2
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080,1


In [12]:
def reorder_playlist(playlist_id, original_index, new_index):
    # Get the current tracks in the playlist
    current_tracks = sp.playlist_tracks(playlist_id)['items']
    current_uris = [track['track']['uri'] for track in current_tracks]
    
    # Rearrange the tracks according to the new index
    new_uris = [current_uris[idx] for idx in new_index]
    
    # Replace the tracks in the playlist with the new order
    sp.playlist_replace_items(playlist_id, new_uris)
    
    # Update the DataFrame with the new indices
    new_order = [i for i in range(len(original_index))]
    df = original_index.to_frame()
    df['new_index'] = new_order

In [13]:
reorder_playlist(uri, df.index, df['new_index'])

In [14]:
# Initialize empty lists for columns
song_names = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

# Loop through the tracks and append the data to the respective lists
for track in sp.playlist_tracks(uri)["items"]:
    track_name = track["track"]["name"]
    song_names.append(track_name)
    
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    danceability.append(audio_features["danceability"])
    energy.append(audio_features["energy"])
    key.append(audio_features["key"])
    loudness.append(audio_features["loudness"])
    mode.append(audio_features["mode"])
    speechiness.append(audio_features["speechiness"])
    acousticness.append(audio_features["acousticness"])
    instrumentalness.append(audio_features["instrumentalness"])
    liveness.append(audio_features["liveness"])
    valence.append(audio_features["valence"])
    tempo.append(audio_features["tempo"])

# Create a DataFrame
data = {
    "Song Name": song_names,
    "Danceability": danceability,
    "Energy": energy,
    "Key": key,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo
}

df = pd.DataFrame(data)
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,"Work REMIX (feat. A$AP Rocky, French Montana, ...",0.841,0.733,1,-5.077,1,0.142,0.024600,0.000000,0.1740,0.284,130.009
1,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080
2,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239
3,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827
4,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013
...,...,...,...,...,...,...,...,...,...,...,...,...
95,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062
96,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983
97,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793
98,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103
